In [2]:
import pandas as pd
import numpy as np
reviews = pd.read_pickle("reviews.pickle")

ratings = reviews[reviews.reviewerID == "A3P51YKG3BCT4K"][["asin","overall"]]
products = pd.read_pickle("products.pickle")
products = products.join(ratings.set_index("asin"))

products_d = products.dropna(subset = ["title","description"])
products_d.dropna(subset=["overall"])

,brand,categories,description,imUrl,price,related,salesRank,title,overall
asin,,,,,,,,,
B000HDJXLW,Muir Glen,[[Grocery & Gourmet Food]],,http://ecx.images-amazon.com/images/I/51SPGYND...,31.00,"{'also_bought': ['B000LKZ9Q4', 'B000LKXFMO', '...",{'Grocery & Gourmet Food': 3429},"Muir Glen Organic Diced Tomatoes, No Salt, 14....",3.0
B000LKXRNQ,Vita Coco,[[Grocery & Gourmet Food]],The Vita Coco brand launched in 2004 and was i...,http://ecx.images-amazon.com/images/I/41yCAH%2...,19.99,"{'also_bought': ['B00CIX6P1U', 'B008JKU2CO', '...",{'Grocery & Gourmet Food': 312},"Vita Coco Coconut Water with Pineapple, 11.1-O...",4.0
B000LKZ9KK,Muir Glen,[[Grocery & Gourmet Food]],,http://ecx.images-amazon.com/images/I/51pLCmCW...,22.99,"{'also_bought': ['B000LKXG64', 'B000LKXFMO', '...",{'Grocery & Gourmet Food': 4654},"Muir Glen Organic Tomatoes, Diced, 14.5-Ounce ...",4.0
B001SB4YPC,WEL PAC,[[Grocery & Gourmet Food]],Wel Pac Noodles Yokogiri Udon is the smoothest...,http://ecx.images-amazon.com/images/I/416Er3CN...,19.50,"{'also_bought': ['B007W0D0HQ', 'B000LKX6R8', '...",{'Grocery & Gourmet Food': 71695},"Wel Pac Noodles Yokogiri Udon, 10-Ounce (Pack ...",3.0
B00374W630,Old El Paso,[[Grocery & Gourmet Food]],Old El Paso Fat Free Refried Beans 16 oz (Pack...,http://ecx.images-amazon.com/images/I/61a6NQWX...,30.89,"{'also_bought': ['B005G2FCI2', 'B005D6E336', '...",{'Grocery & Gourmet Food': 8510},"Old El Paso Refried Beans, Fat Free, 16-Ounce ...",3.0


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
tf = TfidfVectorizer(stop_words="english")
dtm = tf.fit_transform(products_d.description + products_d.title)


from sklearn.metrics.pairwise import cosine_similarity
sim = cosine_similarity(dtm)
sims = sim[:,np.array(products_d.overall.notnull())] #这个人选择这五个商品的similarities
pred = sims.dot(products_d.overall.dropna())
products_d["pred"] = pred

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
dtm #这个dtm表示一共有18835个特征值(词袋)，8470个items.

<8470x18835 sparse matrix of type '<class 'numpy.float64'>'
	with 345854 stored elements in Compressed Sparse Row format>

In [35]:
profile = dtm.T.dot(products_d.overall.fillna(0))

In [36]:
profile.argsort()

array([    0, 12539, 12538, ...,  5533, 11387,  7822])